In [ ]:
import mne
%matplotlib inline

data_file='/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003483/sub-009/ses-1/meg/sub-009_ses-1_task-deduction_run-1_meg.fif'
#(shows events in both cases pretty accurate on this one. average for both is similar)

#data_file='/Volumes/M2_DATA/MEG_QC_stuff/data/from lab/ck06an19_vp13/190521/vp13_block1-1.fif'
#(also mostly ok, some false positives. average for both is similar)

#data_file='/Volumes/M2_DATA/MEG_QC_stuff/data/from lab/forrest_gump_meg/en04ns31_vp15/190524/vp15_block1-1.fif'
#(mostly ok not all). average event is a mess, but equal in both cases


#data_file='/Volumes/M2_DATA/MEG_QC_stuff/data/from lab/Nilo/sub-01_run-postBase_meg.fif/'
#(bad. average nothing in both  cases)


#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003703/sub-a68d5xp5/meg/sub-a68d5xp5_task-listeningToSpeech_run-01_meg.fif'
#(bad. average epoch looks like nothing in both cases. smth is wrong with data?)



raw = mne.io.read_raw_fif(data_file)

picks_ECG = mne.pick_types(raw.info, ecg=True)
ecg_ch=raw.info['chs'][picks_ECG[0]]['ch_name']
print(ecg_ch)

raw



In [ ]:
import plotly.graph_objects as go
import numpy as np

raw_cropped = raw.copy()
tmin_my_plot=400
tmax_my_plot=460
tmin_mne=0
tmax_mne=900
duration = tmax_my_plot-tmin_my_plot
raw_cropped.crop(tmin=tmin_my_plot, tmax=tmax_my_plot)

raw_cropped.copy().pick_types(meg=False, stim=False,ecg=True).plot()


In [ ]:

ecg_events, ch_ecg, average_pulse, ecg=mne.preprocessing.find_ecg_events(raw_cropped, return_ecg=True, verbose=False)
print('Number  of ECG events detected with ECG channel present: ', len(ecg_events))


#Plot the found ECG events on one of the channels:
mag_ch_names = raw.copy().pick_types(meg='mag').ch_names if 'mag' in raw else None
grad_ch_names = raw.copy().pick_types(meg='grad').ch_names if 'grad' in raw else None
channels = {'mag': mag_ch_names, 'grad': grad_ch_names}

data_mag=raw.get_data(picks = channels['mag'])
data_grad=raw.get_data(picks = channels['grad'])

one_data_channel = data_mag[10]

dt=duration/len(one_data_channel) #take 1 random mag channel, for example 10

t=np.arange(tmin_my_plot, tmax_my_plot, 1/raw.info['sfreq'])

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=one_data_channel, name='data'));


fig.update_layout(
    title={
    'text': "ECG events on data of 1 magnetometer",
    'y':0.85,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    xaxis_title="Time in sec",
    yaxis = dict(
        showexponent = 'all',
        exponentformat = 'e'))
        
ecg_events_times  = (ecg_events[:, 0] - raw.first_samp) / raw.info['sfreq']

for ecg_t in ecg_events_times:
    fig.add_vline(x=ecg_t, line_width=1, line_color="green", name="With ecg channel")





In [ ]:
raw_cropped2 = raw.copy()
raw_cropped2.crop(tmin=tmin_my_plot, tmax=tmax_my_plot)

raw_cropped2.drop_channels(ecg_ch)

ecg_events2, ch_ecg2, average_pulse2, ecg=mne.preprocessing.find_ecg_events(raw_cropped2, return_ecg=True, verbose=False)
print('Number of ECG events detected - ECG reconstructed: ', len(ecg_events2))

ecg_events_times2  = (ecg_events2[:, 0] - raw.first_samp) / raw.info['sfreq']

for ecg_t in ecg_events_times2:
    fig.add_vline(x=ecg_t, line_width=1, line_color="red", line_dash="dash", name="Reconstructed")

fig.show()


In [ ]:
#%matplotlib qt

raw_cropped = raw.copy()
raw_cropped.crop(tmin=tmin_mne, tmax=tmax_mne)

raw_cropped2 = raw.copy()
raw_cropped2.crop(tmin=tmin_mne, tmax=tmax_mne)
raw_cropped2.drop_channels(ecg_ch)

ecg_epochs = mne.preprocessing.create_ecg_epochs(raw_cropped)
fig_ecg = ecg_epochs.plot_image(combine='mean') #plot average over ecg epochs artifact

ecg_epochs2 = mne.preprocessing.create_ecg_epochs(raw_cropped2)
fig_ecg2 = ecg_epochs.plot_image(combine='mean') #plot average over ecg epochs artifact


In [ ]:
# Now find which epochs might be more ecg contamminated. 
# Plot the amplitude of all ecg events in 1 channel. 
# For every channel compare each time point of ecg epoch.
# On the data with ECGchannel.

# Thought: even if we get that one epoch in this method is more contamminated - 
# data amplitudes are higher - it doesnt really speak for ecg contammination, maybe it s just a noisy epoch?



data_ecg_eps_mag = ecg_epochs.get_data(picks='mag')
#gives array of shape (n_epochs, n_channels, n_times)
#ecg_epochs.get_data(picks='mag')[0][0] # first epoch, first channel

t_ep=(np.arange(-len(data_ecg_eps_mag[0][0])/2, len(data_ecg_eps_mag[0][0])/2))/1000
fig = go.Figure()

ch_peak_loc_all=[]
ch_peak_mag_all=[]
for ep in [data_ecg_eps_mag[4]]:
    for ind_ch, ch_data in enumerate(ep[0:3]):
        fig.add_trace(go.Scatter(x=t_ep, y=ch_data, name='Ch_#'+str(ind_ch)));

        #detect peaks of the epoch data of 1 channel (absolute values, cos we r onlyinterested in amplitudes,  not in -/+)
        ch_peak_loc, ch_peak_mag = mne.preprocessing.peak_finder(np.abs(ch_data), thresh=1e-13, extrema=1)
        # ch_peak_loc_all.append(ch_peak_loc) if ch_peak_loc else ch_peak_loc_all.append(None)
        # ch_peak_mag_all.append(ch_peak_mag) if ch_peak_mag  else ch_peak_mag_all.append(None)

    first_3_ch = ep[0:3]

fig.update_layout(
    title={
    'text': "1 ECG epoch: data for 3 mag",
    'y':0.85,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    xaxis_title="Time in s",
    yaxis = dict(
        showexponent = 'all',
        exponentformat = 'e'))
fig.show()

In [ ]:
first_3_ch

In [ ]:
#Unrelatrd to ecg itself. How to save mtplotlib as html string:
import base64
from io import BytesIO


#plot sth

tmpfile = BytesIO()
fig_ecg[0].savefig(tmpfile, format='png', dpi=130) #writing image into a temporary file
encoded = base64.b64encode(tmpfile.getvalue()).decode('utf-8')

html = 'Some html head' + '<img src=\'data:image/png;base64,{}\'>'.format(encoded) + 'Some more html'

with open('ecg_fig.html','w') as f:
    f.write(html)